In [1]:
import tensorflow as tf

import numpy as np
import os
import time
print(tf.__version__)

2.7.0


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [6]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [7]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [8]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [9]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [10]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)


In [11]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [12]:
dataset = sequences.map(split_input_target)

In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 2048



In [15]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
#        self.lstm = tf.keras.layers.LSTM(150,return_sequences=True)
#        self.denseextra = tf.keras.layers.Dense(500, activation="relu")        
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
#        x = self.lstm(x,training=training)
#        x = self.denseextra(x, training=training)        
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [16]:
#model.add(Bidirectional(LSTM(150, return_sequences = True)))# An LSTM Layer)
#model.add(Dropout(0.2))# A dropout layer)
#model.add(LSTM(100))# Another LSTM Layer)
#model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))# A Dense Layer including regularizers)
#model.add(Dense(total_words, activation='softmax'))# A Dense Layer)

In [17]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  14168064  
                                                                 
 dense (Dense)               multiple                  135234    
                                                                 
Total params: 14,320,194
Trainable params: 14,320,194
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [21]:
sampled_indices
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ure,\nWhich easily endures not article\nTying him to aught; so putting him to rage,\nYou should have ta'

Next Char Predictions:
 b'utOYzAPFpieFbCmehK\nJC,b&t&DnRGQAOS\n;3RvxTuQWWlVqgOXqwbTQFxSaI\nC[UNK]-zNmlLIrUv-wn$Lii hK[UNK]!-hbh[UNK]iUyg!M?Vq'


In [22]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [23]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.1892266


In [24]:
tf.exp(mean_loss).numpy()

65.97175

In [25]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [27]:
EPOCHS = 20

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 21s 113ms/step - loss: 2.7202 - accuracy: 0.3068
Epoch 2/20
172/172 [==============================] - 20s 113ms/step - loss: 1.9070 - accuracy: 0.4403
Epoch 3/20
172/172 [==============================] - 20s 113ms/step - loss: 1.6347 - accuracy: 0.5129
Epoch 4/20
172/172 [==============================] - 20s 113ms/step - loss: 1.4735 - accuracy: 0.5552
Epoch 5/20
172/172 [==============================] - 20s 113ms/step - loss: 1.3708 - accuracy: 0.5812
Epoch 6/20
172/172 [==============================] - 20s 113ms/step - loss: 1.2909 - accuracy: 0.6009
Epoch 7/20
172/172 [==============================] - 20s 113ms/step - loss: 1.2181 - accuracy: 0.6197
Epoch 8/20
172/172 [==============================] - 20s 113ms/step - loss: 1.1430 - accuracy: 0.6401
Epoch 9/20
172/172 [==============================] - 20s 113ms/step - loss: 1.0584 - accuracy: 0.6644
Epoch 10/20
172/172 [==============================] - 20s 113ms/step - l

In [29]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [30]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [31]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The swinness of his hope, he took'd his neighbours:
Their love painted man that I was fonger, they are
They one Master than you part.

POLIXENES:
My father's son, awake;
Let them have a bitter scoff; men their breadhs,
Which birshy parted but a little perio!
The Earl of Pisa, sir, having
may sin
And dares this day and so have
sent out a word. But what talk sprites as true?

BAPTISTA:
Many an old man shall we two hung from hence.
Boisley as I am, would I were foreward with;
Which frails am nothing, not no whit leisure;
O, his depart, bearing moved me, to quicker thee.

ROMEO:
Have done, my lord, Mistress Bishop
That Bolingbroke was not his bag born.

ANGELO:
Now, sir, his name is Lucentio,' quoth he, 'poating with,
Since princes not so quick condial with foreign
And tell false tribunes to his sovereign liege,
And fall somewhat with a doubtly guilt!
Somewhore the priest that I was no near prononn.

KING RICHARD II:
Cousin, throw up your grace?

GLOUCESTER:
Be a horse, by God, his 